In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(r"ny_od_main_JT00_2010.csv")

In [3]:
df = df.drop(columns=['createdate'])
df.head()

,w_geocode,h_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03
0,360010001001004,360010001001009,1,0,1,0,0,0,1,1,0,0
1,360010001001004,360010001001010,1,0,1,0,0,0,1,1,0,0
2,360010001001004,360010001001033,1,0,1,0,0,0,1,1,0,0
3,360010001001004,360010014004001,1,0,1,0,0,0,1,0,1,0
4,360010001001004,360010015001007,1,0,1,0,0,0,1,0,1,0


In [4]:
df['w_tract'] = df['w_geocode'].astype(str).str[:11].astype(int)
df['h_tract'] = df['h_geocode'].astype(str).str[:11].astype(int)

w_geocode: workplace census tract code

h_geocode: home census tract code

S000: total number of jobs

SA01: number of workers age 29 or younger

SA02: number of workers age 30-54

SA03: number of workers age 55 or older

SE01: number of jobs with earnings 1250 or less

SE02: number of jobs with earnings 1251-3333

SE03: number of jobs with earnings above 3334

SI01: number of jobs in goods producing industry sectors

SI02: number of jobs in Trade, Transportation and Utilities

SI03: number of jobs in All Other Services

In [5]:
df_agg = df.groupby(['w_tract', 'h_tract']).sum().reset_index().drop(columns=['w_geocode', 'h_geocode'])

Central county in Albany OMB defined MSA: Albany County, Rensselaer County, Saratoga County, Schenectady County

Tracts in Albany County, Rensselaer County, Saratoga County, Schenectady County:
* Albany county: 36001
* Rensselaer county: 36083
* Saratoga county: 36091
* Schenectady county: 36093

Central county in Syracuse OMB defined MSA: 
* Onondaga County 36067

In [ ]:
df_agg['w_county'] = df_agg['w_tract'].astype(str).str[:5].astype(int)
df_agg['h_county'] = df_agg['h_tract'].astype(str).str[:5].astype(int)
df_agg.head()

,w_tract,h_tract,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,w_county,h_county
0,36001000100,36001000100,29,5,18,6,4,10,15,5,15,9,36001,36001
1,36001000100,36001000201,11,2,7,2,3,6,2,0,8,3,36001,36001
2,36001000100,36001000202,5,1,4,0,0,5,0,0,4,1,36001,36001
3,36001000100,36001000301,3,2,1,0,0,2,1,0,1,2,36001,36001
4,36001000100,36001000302,10,3,6,1,2,7,1,1,3,6,36001,36001


In [ ]:
df_agg['w_albany_core'] = df_agg['w_county'].apply(lambda x: 1 if x in [36001, 36083, 36091, 36093] else 0)
df_agg['h_albany_core'] = df_agg['h_county'].apply(lambda x: 1 if x in [36001, 36083, 36091, 36093] else 0)
df_agg['w_syracuse_core'] = df_agg['w_county'].apply(lambda x: 1 if x in [36067] else 0)
df_agg['h_syracuse_core'] = df_agg['h_county'].apply(lambda x: 1 if x in [36067] else 0)

In [8]:
albany_w_tracts = df_agg.query('w_albany_core==1')['w_tract'].unique()
albany_h_tracts = df_agg.query('h_albany_core==1')['h_tract'].unique()

In [9]:
#sanity check
assert len(albany_h_tracts) == len(albany_w_tracts), "AssertionError: the number of w and h tracts don't match"
print(len(albany_h_tracts))

85


In [10]:
syracuse_w_tracts = df_agg.query('w_syracuse_core==1')['w_tract'].unique()
syracuse_h_tracts = df_agg.query('h_syracuse_core==1')['h_tract'].unique()

In [11]:
#sanity check
assert len(syracuse_h_tracts) == len(syracuse_w_tracts), "AssertionError: the number of w and h tracts don't match"
print(len(syracuse_w_tracts))

142


In [12]:
df_agg_s = df_agg[['w_tract', 'h_tract', 'S000', 'w_albany_core', 'h_albany_core', 'w_syracuse_core', 'h_syracuse_core']]

In [13]:
total_worker_per_w_tract = df_agg_s[['w_tract', 'S000']].groupby('w_tract').sum().reset_index()
total_worker_per_h_tract = df_agg_s[['h_tract', 'S000']].groupby('h_tract').sum().reset_index()

In [14]:
df_agg_s1 = pd.merge(df_agg_s, total_worker_per_w_tract, left_on='w_tract', right_on='w_tract', how='inner')
df_agg_s2 = pd.merge(df_agg_s1, total_worker_per_h_tract, left_on='h_tract', right_on='h_tract', how='inner')

In [15]:
df_agg_s2.rename(columns={'S000_x': 'S000', 'S000_y': 'S000_w_total', 'S000': 'S000_h_total'}, inplace=True)

In [16]:
df_agg_s2['w_share'] = df_agg_s2['S000'] / df_agg_s2['S000_w_total']
df_agg_s2['h_share'] = df_agg_s2['S000'] / df_agg_s2['S000_h_total']

In [17]:
df_agg_s2 
#example: on line 2, there are 11 workers who go from tract 3601000201 to tract 36001000100. They represent 0.5% of the 
#workers who live in tract 36001000100 and 0.1% of the workers who work in tract 3601000201

,w_tract,h_tract,S000,w_albany_core,h_albany_core,w_syracuse_core,h_syracuse_core,S000_w_total,S000_h_total,w_share,h_share
0,36001000100,36001000100,29,1,1,0,0,2051,853,0.014139,0.033998
1,36001000100,36001000201,11,1,1,0,0,2051,1108,0.005363,0.009928
2,36001000100,36001000202,5,1,1,0,0,2051,998,0.002438,0.005010
3,36001000100,36001000301,3,1,1,0,0,2051,1329,0.001463,0.002257
4,36001000100,36001000302,10,1,1,0,0,2051,1029,0.004876,0.009718
...,...,...,...,...,...,...,...,...,...,...,...
2150955,36123150502,36123150301,3,0,0,0,0,242,1010,0.012397,0.002970
2150956,36123150502,36123150302,6,0,0,0,0,242,825,0.024793,0.007273
2150957,36123150502,36123150400,17,0,0,0,0,242,1246,0.070248,0.013644
2150958,36123150502,36123150501,21,0,0,0,0,242,744,0.086777,0.028226


In [18]:
def attach_tracts_to_core(df):
    albany_attach_h_tracts = (df.query('w_albany_core==1 & h_share >= 0.25')['h_tract']).unique() 
    albany_attach_w_tracts = (df.query('h_albany_core==1 & w_share >= 0.25')['w_tract']).unique()
    df_prime = df.copy()
    df_prime['w_albany_core'] = df_prime['w_albany_core'] + ~(df_prime['w_albany_core'].astype(bool)) * df_prime['w_tract'].apply(lambda x: 1 if x in albany_attach_w_tracts else 0)
    df_prime['h_albany_core'] = df_prime['h_albany_core'] + ~(df_prime['h_albany_core'].astype(bool)) * df_prime['h_tract'].apply(lambda x: 1 if x in albany_attach_h_tracts else 0)
    return df_prime


In [24]:
df_agg_s2.query('h_albany_core==1 & w_share >= 0.25')

,w_tract,h_tract,S000,w_albany_core,h_albany_core,w_syracuse_core,h_syracuse_core,S000_w_total,S000_h_total,w_share,h_share
29917,36001013802,36001013802,28,1,1,0,0,95,2484,0.294737,0.011272
38560,36001014402,36001014402,120,1,1,0,0,356,1595,0.337079,0.075235
43372,36001014700,36001014700,28,1,1,0,0,51,1217,0.549020,0.023007
43418,36001014801,36001014801,167,1,1,0,0,426,1399,0.392019,0.119371
43469,36001014802,36001014802,84,1,1,0,0,184,737,0.456522,0.113976
43527,36001014803,36001014803,147,1,1,0,0,296,1548,0.496622,0.094961


In [32]:
df_agg_s2.query('w_albany_core != h_albany_core')['S000'].value_counts()

S000
1      38127
2       9777
3       4279
4       2325
5       1467
       ...  
126        1
69         1
79         1
91         1
85         1
Name: count, Length: 120, dtype: int64

,w_tract,h_tract,S000,w_albany_core,h_albany_core,w_syracuse_core,h_syracuse_core,S000_w_total,S000_h_total,w_share,h_share
0,36001000100,36001000100,29,1,1,0,0,2051,853,0.014139,0.033998
1,36001000100,36001000201,11,1,1,0,0,2051,1108,0.005363,0.009928
2,36001000100,36001000202,5,1,1,0,0,2051,998,0.002438,0.005010
3,36001000100,36001000301,3,1,1,0,0,2051,1329,0.001463,0.002257
4,36001000100,36001000302,10,1,1,0,0,2051,1029,0.004876,0.009718
...,...,...,...,...,...,...,...,...,...,...,...
2150955,36123150502,36123150301,3,0,0,0,0,242,1010,0.012397,0.002970
2150956,36123150502,36123150302,6,0,0,0,0,242,825,0.024793,0.007273
2150957,36123150502,36123150400,17,0,0,0,0,242,1246,0.070248,0.013644
2150958,36123150502,36123150501,21,0,0,0,0,242,744,0.086777,0.028226


In [ ]:
##we want to attach to the core all tracts who have 25% or more of their workers going to or from the core
df_agg_s2.query('w_albany_core==1').query('h_share >= 0.25')['h_tract'] ##lists the h_tracts such that +25% of them work in a core albany


14939      36001002100
1690361    36083051702
Name: h_tract, dtype: int64

In [34]:
~df_agg_s2['h_albany_core'].astype(bool)

0          False
1          False
2          False
3          False
4          False
           ...  
2150955     True
2150956     True
2150957     True
2150958     True
2150959     True
Name: h_albany_core, Length: 2150960, dtype: bool